In [30]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [31]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)

In [32]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [33]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [34]:
line = Line([y, z])

In [35]:
eqn_SE1 = P(cubic_new.subs({b:-(c*c+e*f)/c }) )
sing_cubics_SE1 = P(sing_cubics.subs({b:-(c*c+e*f)/c }))

In [36]:
eqn_SE3 = P(2*eqn_SE1.subs({d:(3*e*f - c^2 + c*f + c*e)/(2*c)}) )
sing_cubics_SE3 = P(sing_cubics_SE1.subs({d:(3*e*f - c^2 + c*f + c*e)/(2*c)}))

In [37]:
eqn_SE6 = P((c+e)*eqn_SE3.subs({f: -c*(5*c+e)/(c+e)})/(4*c))
sing_cubics_SE6 = P((c+e)^6*sing_cubics_SE3.subs({f: -c*(5*c+e)/(c+e)}))

In [38]:
SE6 = Cubic(eqn_SE6, line, sing_cubics_SE6.factor())

In [39]:
SE6.eckardt_points

[(0, 0, 0, 1),
 (0, -c^2 - c*e, 2*c^2 - 3*c*e - e^2, 0),
 (0, 1, 1, 0),
 (0, 0, 1, -1),
 (0, c^2 + c*e, -2*c^2 + 3*c*e + e^2, 3*c^2 - 2*c*e - e^2),
 (0, c^2 + c*e, c^2 + c*e, -3*c^2 + 2*c*e + e^2)]

In [40]:
def apply_proj_to_eck(proj, eck):
    new_indices = []
    for i in range(len(eck)):
        new_indices.append(eck.index(eck[i]*proj)+1)
    return new_indices

In [42]:
labels = [pl.labels for pl in SE6.tritangent_planes if pl.conditions == 0]
keys = list(SE6.cl_lines.keys())
labels

[['E1', 'G4', 'F14'],
 ['E2', 'G1', 'F12'],
 ['E2', 'G3', 'F23'],
 ['E3', 'G4', 'F34'],
 ['E4', 'G2', 'F24'],
 ['F13', 'F24', 'F56']]

In [43]:
adm_perm = SE6.find_admissible_permutations()

In [44]:
[adm_perm[0][keys.index(label)] for label in labels[3]]

['E3', 'G4', 'F34']

In [45]:
print(adm_perm[0])

['E1', 'E2', 'E3', 'E4', 'E6', 'E5', 'G1', 'G2', 'G3', 'G4', 'G6', 'G5', 'F12', 'F13', 'F14', 'F16', 'F15', 'F23', 'F24', 'F26', 'F25', 'F34', 'F36', 'F35', 'F46', 'F45', 'F56']


In [47]:
resulting_L_sets = []
conjugate_perm = []
for perm in adm_perm:
    perm_L_set = [perm[list(SE6.cl_lines.keys()).index(label)] for label in ['E1', 'G4', 'E2', 'G3', 'E3']]
    if perm_L_set not in resulting_L_sets:
        resulting_L_sets.append(perm_L_set)
    else:
        conjugate_perm.append(perm)

In [48]:
len(conjugate_perm)

48

In [49]:
adm_proj = SE6.find_admissible_projectivities()

In [50]:
simm_SE6 = SE6.find_simmetries(adm_proj)

In [52]:
len(adm_proj)

48

In [69]:
perms = []
normal_perm = [perm for perm in adm_perm if perm not in conjugate_perm]
print(len(normal_perm))
for simm in simm_SE6:
    perm = normal_perm[adm_proj.index(simm)]
    perms.append(Permutation([perm.index(label)+1 for label in keys]).to_cycles())
    #perm = conjugate_perm[adm_proj.index(simm)]
    #perms.append(Permutation([perm.index(label)+1 for label in keys]).to_cycles())

48


In [70]:
len(perms)

24

In [62]:
Gr = PermutationGroup(perms)

In [74]:
subgr = []
for subg in Gr.subgroups():
    if subg.order() == 24:
        subgr.append(subg)

In [129]:
subgr[0].gens()[0].dict()

{1: 1,
 2: 19,
 3: 22,
 4: 7,
 5: 25,
 6: 26,
 7: 4,
 8: 13,
 9: 14,
 10: 10,
 11: 16,
 12: 17,
 13: 8,
 14: 9,
 15: 15,
 16: 11,
 17: 12,
 18: 27,
 19: 2,
 20: 24,
 21: 23,
 22: 3,
 23: 21,
 24: 20,
 25: 5,
 26: 6,
 27: 18}

In [145]:
L_set = []
for key in [1, 10, 2,9,3]:
    L_set.append(keys[subgr[0].gens()[1].dict()[key]])

In [146]:
L_set

['E4', 'G5', 'E3', 'F24', 'E2']

In [147]:
find_all_proj_parallel(SE6.cl_lines, [L_set]) in simm_SE6

False